In [ ]:
import numpy
import plotly
from sklearn import linear_model
from sklearn import datasets

In [ ]:
diabetes = dtasets.load_diabetes()
x = diabetes.data
y = diabetes.target

In [ ]:
beta = len()